### assess scores using mubind

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import mubind
import pandas as pd
import glob
import os
import anndata


In [4]:
cd ~/workspace/theislab/mubind-pipeline/notebooks/pipeline

/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/mubind-pipeline/notebooks/pipeline


In [5]:
import utils

In [6]:
import torch
import mubind as mb

rootdir = '/mnt/f/workspace/theislab/mubind/data/*/*/mubind_output/obs*/*'
overwrite = False

df = []
for p in glob.glob(rootdir):

    print(p)
    continue

    if not os.path.exists(p):
        continue
    
    if not 'model_0.pt' in p:
        continue

    scores_path = p.replace('.pt', '_scores.csv')
    print(os.path.exists(scores_path), scores_path)
    if os.path.exists(scores_path) and not overwrite:
        df2 = pd.read_csv(scores_path, index_col=0)
        df2['n_obs'] = int(scores_path.split('/')[-2].replace('obs', ''))
        df.append(df2)
        continue

    res = []

    model = torch.load(p)

    print(p)

    os.makedirs("results", exist_ok=True)

    n_obs = p.split('/')[-2]
    dataset_name = p.split('/')[-4]

    ad_path = os.path.join(os.path.dirname(p),
                            '../../scbasset_input/%s/ad.h5ad' % n_obs)
    splits_path = os.path.join(os.path.dirname(p),
                                '../../scbasset_input/%s/splits.h5' % n_obs)
    peaks_path = os.path.join(os.path.dirname(p),
                                '../../scbasset_input/%s/peaks.bed' % n_obs)
    print(os.path.exists(ad_path), ad_path)

    # read h5ad file
    ad = anndata.read_h5ad(ad_path)

    print('preparing sequences test...')
    dataset_name = p.split('/')[-4]
    print(dataset_name)

    species = 'hg38' if (dataset_name != 'noack_2022' and dataset_name != 'pancreatic_endocrinogenesis') else 'mm10'
    gen_path = '/mnt/c/Users/ignacio.ibarra/Dropbox/annotations/%s/genome/%s.fa' % (species, species)

    seqs = pd.read_csv(peaks_path, sep='\t', header=None)
    seqs.columns = ['chr', 'start', 'end']
    print(seqs.head())
    all_seqs = mb.bindome.tl.get_sequences_from_bed(seqs[['chr', 'start', 'end']],
                                                    gen_path=gen_path,
                                                    uppercase=True)
                                                # gen_path='../../../annotations/dm6/genome/dm6.fa')
    keys_test = set([s[0] for s in all_seqs])
    # adata_test = adata_test[:,adata_test.var['k_summit'].isin(keys_test)]

    # convert to csr matrix
    import h5py
    with h5py.File(splits_path, 'r') as hf:
        train_ids = hf['train_ids'][:]
        val_ids = hf['val_ids'][:]
        test_ids = hf['test_ids'][:]


    assert ad.shape[1] == len(all_seqs)
    # print(adata_train.shape, adata_test.shape)


    import numpy as np
    import torch.utils.data as tdata

    res = []

    ad_train = ad[:,train_ids]
    ad_val = ad[:,val_ids]
    ad_test = ad[:,test_ids]
    ad_train.shape, ad_val.shape, ad_test.shape

    all_seqs = np.array(all_seqs)
    train_seqs, val_seqs, test_seqs = all_seqs[train_ids], all_seqs[val_ids], all_seqs[test_ids]

    for next_ad, next_seqs, next_label in zip([ad_train, ad_val, ad_test],
                                            [train_seqs, val_seqs, test_seqs],
                                            ['train', 'val', 'test']):
        next_df = utils.prepare_df(next_seqs, next_ad)

        dataset = mb.datasets.SelexDataset(next_df,
                                                n_rounds=next_df.shape[1],
                                                enr_series=False)
        dataloader = tdata.DataLoader(dataset=dataset,
                                batch_size=512,
                                shuffle=True)

        roc_auc = utils.get_auroc(model, dataloader)
        scores = mb.tl.scores(model, dataloader)

        res.append([dataset_name, next_label, 'roc_auc', roc_auc])
        res.append([dataset_name, next_label, 'r2', scores['r2_counts']])

    res = pd.DataFrame(res, columns=['dataset', 'group', 'metric', 'value'])
    res.to_csv(scores_path)
    
    df.append(res)

res = pd.concat(df)
res['model'] = 'mubind'
print('')

/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/mubind_output/obs2000/model_0.pt
/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/mubind_output/obs2000/model_1.pt
/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/mubind_output/obs2000/model.csv
/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/mubind_output/obs2000/model_0_scores.csv
/mnt/f/workspace/theislab/mubind/data/noack_2022/mubind_output/obs500/model.csv
/mnt/f/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/mubind_output/obs1000/model.csv
/mnt/f/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/mubind_output/obs1000/model_0.pt
/mnt/f/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/mubind_output/obs1000/model_1.pt
/mnt/f/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/mubind_output/obs1000/model_0_scores.csv


ValueError: No objects to concatenate

In [8]:
res

,dataset,group,metric,value,n_obs,model
0,organoids_treutlein_dataset,train,roc_auc,0.767851,2000,mubind
1,organoids_treutlein_dataset,train,r2,0.000809,2000,mubind
2,organoids_treutlein_dataset,val,roc_auc,0.734101,2000,mubind
3,organoids_treutlein_dataset,val,r2,0.000292,2000,mubind
4,organoids_treutlein_dataset,test,roc_auc,0.727061,2000,mubind
5,organoids_treutlein_dataset,test,r2,0.001874,2000,mubind
0,pancreatic_endocrinogenesis,train,roc_auc,0.646011,1000,mubind
1,pancreatic_endocrinogenesis,train,r2,0.001432,1000,mubind
2,pancreatic_endocrinogenesis,val,roc_auc,0.626660,1000,mubind
3,pancreatic_endocrinogenesis,val,r2,0.000992,1000,mubind


In [7]:
res.to_csv('results_mubind.csv')